<a href="https://colab.research.google.com/github/BillWorstell/derenzo_phantom/blob/master/CollectiveDiffDRR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate 3D homogeneous projections from (Euclidean) volume images of a derenzo_phantom

In [1]:
#Install os and sys, mount drive
import os
import sys
from google.colab import drive
#drive.mount('/content/drive')

!pip install icecream
from icecream import ic

import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpp

In [ ]:
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("2.1.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-cvyky0a8
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-cvyky0a8
  Running command git checkout -q 2f11ddc5ee7d6bd56f2fb6744a16776fab6536f7
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 2f11ddc5ee7d6bd56f2fb6744a16776fab6536f7
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Load data

#Mount google drive and connect to output data path

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

##Load translations and rotations of phantom

In [ ]:
RunTranslate=np.load('/content/gdrive/MyDrive/data/derenzo_phantom/RunTranslate.npy')
ic(RunTranslate.shape)
NRuns=RunTranslate.shape[0]
ic(NRuns)

RunRotate=np.load('/content/gdrive/MyDrive/data/derenzo_phantom/RunRotate.npy')
ic(RunRotate.shape)
NRuns=RunRotate.shape[0]
ic(NRuns)

##Load ground truth volumes orthogonal projections

In [ ]:
NEvents=1

for iEvent in range(0,NEvents):
  ic(iEvent)
  thisTruth=torch.load(f'/content/gdrive/MyDrive/data/derenzo_phantom/truth_{iEvent}.pt')
  Truth=thisTruth.numpy()
  Truth_Mod0_ZSum=torch.sum(thisTruth,dim=3).numpy()
  Truth_Mod0_YSum=torch.sum(thisTruth,dim=2).numpy()
  Truth_Mod0_XSum=torch.sum(thisTruth,dim=1).numpy()

  TX=RunTranslate[iEvent,0]
  TY=RunTranslate[iEvent,1]
  TZ=RunTranslate[iEvent,2]
  PsiDeg=(180./np.pi)*RunRotate[iEvent,0]
  ThetaDeg=(180./np.pi)*RunRotate[iEvent,1]
  PhiDeg=(180./np.pi)*RunRotate[iEvent,2]

  plt.imshow(np.flipud(Truth_Mod0_ZSum.T),cmap='gray')
  plt.title(f'Event{iEvent} Truth ZSum')
  plt.xlabel(f'Event{iEvent} TX={TX:5.1f} TY={TY:5.1f} TZ={TZ:5.1f}')
  plt.ylabel(f'Event{iEvent} Theta={ThetaDeg:.0f} Phi={PhiDeg:.0f}')
  plt.show()

  plt.imshow(np.flipud(Truth_Mod0_YSum.T),cmap='gray')
  plt.title(f'Event{iEvent} Truth YSum')
  plt.xlabel(f'Event{iEvent} TX={TX:5.1f} TY={TY:5.1f} TZ={TZ:5.1f}')
  plt.ylabel(f'Event{iEvent} Theta={ThetaDeg:.0f} Phi={PhiDeg:.0f}')
  plt.show()

  plt.imshow(np.flipud(Truth_Mod0_XSum.T),cmap='gray')
  plt.title(f'Event{iEvent} Truth XSum')
  plt.xlabel(f'Event{iEvent} TX={TX:5.1f} TY={TY:5.1f} TZ={TZ:5.1f}')
  plt.ylabel(f'Event{iEvent} Theta={ThetaDeg:.0f} Phi={PhiDeg:.0f}')
  plt.show()


#Get Multi-pinhole SPECT System Geometry

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/SPECTGeometry/'

In [ ]:
!ls -ltr /content/drive/MyDrive/SPECTGeometry/

Install openpyxl using pip

In [ ]:
pip install openpyxl

https://openpyxl.readthedocs.io/en/stable/tutorial.html#loading-from-a-file

In [ ]:
from openpyxl import load_workbook
wb = load_workbook(filename = '/content/drive/MyDrive/SPECTGeometry/MDSL.excel80M10RFR.cut-plate.008.150roi.2.30pin.105ellipse.xlsx',data_only=True)

loop through worksheets

In [ ]:
for sheet in wb:
...     print(sheet.title)

Go to Coordinates Worksheet

In [ ]:
wb.active = 1
print(wb.active.title)
ws = wb.active

X,Y,Z Coordinates at the center of each pinhole

In [ ]:
ic(ws.cell(2,7).value)
XP=np.zeros(80)
YP=np.zeros(80)
ZP=np.zeros(80)
for i in range(3,83):
  XP[i-3]=(ws.cell(i,2).value)
  YP[i-3]=(ws.cell(i,3).value)
  ZP[i-3]=(ws.cell(i,4).value)

ic(XP[0])
ic(YP[0])
ic(ZP[0])

Length of Collimator

In [ ]:
ic(ws.cell(2,7).value)
lcoll=np.zeros(80)
for i in range(3,83):
  lcoll[i-3]=(ws.cell(i,7).value)

Choose vsdr = 5 * length of collimator, so by similar triangles

(2*5*lcoll)=source to virtual detector, or 10X

source to virtual detector is 10X size of actual detector

Size of detector at end of collimator is ~50mm

This implies dx * NX = 10 * 50mm ~ 500mm

dx ~ 500/256 ~2mm

    height=256,  # Height of the DRR (if width is not seperately provided, the generated image is square)
    delx=1.0,  # Pixel spacing (in mm)


In [ ]:
vsdr=5.*lcoll

alpha: Azimuthal angle (radians)

In [ ]:
ic(ws.cell(2,22).value)
alpha=np.zeros(80)
for i in range(3,83):
  alpha[i-3]=(ws.cell(i,22).value)

beta = altitude (radians)

In [ ]:
ic(ws.cell(2,23).value)
beta=np.zeros(80)
for i in range(3,83):
  beta[i-3]=(ws.cell(i,23).value)

#make meshgrid from volume and spacing

In [ ]:
#volume, spacing = load_example_ct()
volume=np.zeros([256,256,256])
spacing=[1.0, 1.0, 1.0]
bx, by, bz = torch.tensor(volume.shape) * torch.tensor(spacing) / 2
ic(bx, by, bz)
ic(volume.shape)
ic(spacing)

Get lab frame coordinates for voxels in volume

In [ ]:
xlinspace=np.linspace(-128*spacing[0], 128*spacing[0], 256)
ylinspace=np.linspace(-128*spacing[1], 128*spacing[1], 256)
zlinspace=np.linspace(-128*spacing[2], 128*spacing[2], 256)
xgrid,ygrid,zgrid = np.meshgrid(xlinspace, ylinspace,zlinspace)
xgrid=xgrid.flatten()
ygrid=ygrid.flatten()
zgrid=zgrid.flatten()

#Define camera[0] using PyTorch3D methods

Import methods from pytorch3d

In [ ]:
# io utils
from pytorch3d.io import load_obj

# datastructures
from pytorch3d.structures import Meshes

# 3D transformations functions
from pytorch3d.transforms import Rotate, Translate

# rendering components
from pytorch3d.renderer import (
    PerspectiveCameras, OrthographicCameras, FoVOrthographicCameras,
    FoVPerspectiveCameras, look_at_view_transform, look_at_rotation,
    RasterizationSettings, MeshRenderer, MeshRasterizer, BlendParams,
    SoftSilhouetteShader, HardPhongShader, PointLights, TexturesVertex,
)

# Map from k-depth extended projection image 3D tensor to 3d volume

#Mount google drive and connect to output data path

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Look around data area

In [ ]:
!ls -ltr /content/gdrive/MyDrive/data/derenzo_phantom/truth_0.pt
!ls -ltr /content/gdrive/MyDrive/data/derenzo_phantom/Projections_0.pt
!ls -ltr /content/gdrive/MyDrive/data/derenzo_phantom/ProjRadon96_0.pt
!ls -ltr /content/gdrive/MyDrive/data/derenzo_phantom/ProjRadon256_0.pt
!ls -ltr /content/gdrive/MyDrive/data/derenzo_phantom/RunTranslate.npy
!ls -ltr /content/gdrive/MyDrive/data/derenzo_phantom/RunRotate.npy

# Set up projective geometry for comparison with diffDRR as called earlier by iDerenzoRandomTomograph.ipynb

Length of Collimator

In [ ]:
ic(ws.cell(2,7).value)
lcoll=np.zeros(80)
for i in range(3,83):
  lcoll[i-3]=(ws.cell(i,7).value)

##Get DiffDRR version from github

Use the version of DiffDRR posted on github

Git clone from github publib open source code to colab working directory

See https://stackoverflow.com/questions/50850216/google-colab-install-from-github-glrm

In [ ]:
#!pip install 'git+https://github.com/BillWorstell/DiffDRR.git'
!pip install 'git+https://github.com/eigenvivek/DiffDRR.git'

Import diffDRR modules

In [ ]:
from diffdrr.data import load_example_ct
from diffdrr.drr import DRR
from diffdrr.visualization import plot_drr
from diffdrr.pose import RigidTransform
from diffdrr.pose import convert

import euler_angles_to_matrix

In [ ]:
#volume, spacing = load_example_ct()
#volume=np.zeros([256,256,256])
spacing=[1.0, 1.0, 1.0]
ic(spacing)

for iRun in range(0,NRuns):
  thisTruth=torch.load(f'/content/gdrive/MyDrive/data/derenzo_phantom/truth_{iEvent}.pt')
  volume = torch.squeeze(thisTruth)
  ic(volume.shape)
  #
  for imod in range(0,2):
    ############################
    # Generate DRRs
    ############################
    #
    #| cuda
    # Read in the volume and get the isocenter
    bx, by, bz = torch.tensor(volume.shape) * torch.tensor(spacing) / 2
    # Initialize the DRR module for generating synthetic X-rays
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #Collect Projections over all modules
    #Projections=torch.zeros([1,96,256,256])

    ic(iRun)
    drr = DRR(
      volume,  # The CT volume as a numpy array
      spacing,  # Voxel dimensions of the CT
      sdr=float(vsdr[imod]),  # Source-to-detector radius (half of the source-to-detector distance)
      height=256,  # Height of the DRR (if width is not seperately provided, the generated image is square)
      delx=2.0,  # Pixel spacing (in mm)
    ).to(device)

    # Set the camera pose with rotations (yaw, pitch, roll) and translations (x, y, z)
    rotations = torch.tensor([[float(alpha[imod]), float(beta[imod]), torch.pi / 2]], device=device)
    translations = torch.tensor([[bx, by, bz]], device=device)
    img = drr(rotations, translations, parameterization="euler_angles", convention="ZYX")

    pose = convert(rotations, translations, parameterization="euler_angles", convention="ZYX")
    source, rays = drr.detector(pose)
    ic(source.shape)
    ic(source)
    ic(rays.shape)

    source_ = source.detach().cpu()
    rays_ = rays.permute(2, 0, 1).detach().cpu()

    if(imod==0):
      Projection_mod0=torch.squeeze(img).cpu().numpy()
    if(imod==1):
      Projection_mod1=torch.squeeze(img).cpu().numpy()


Display Event0 Mod0 Projection

In [ ]:
ic(Projection_mod0.shape)
plt.imshow(np.flipud(Projection_mod0.T),cmap='gray')

#  ThisAspect=24/256
#  plt.imshow(np.flipud(eventProjections_2DZSumRow2.T),cmap='gray',aspect=ThisAspect)
plt.title(f'Event{iEvent} Projection_mod0')
plt.xlabel(f'Event{iEvent} TX={TX:5.1f} TY={TY:5.1f} TZ={TZ:5.1f}')
plt.ylabel(f'Event{iEvent} Theta={ThetaDeg:.0f} Phi={PhiDeg:.0f}')
plt.show()


https://vivekg.dev/DiffDRR/tutorials/optimizers.html

In [ ]:
from diffdrr.utils import make_intrinsic_matrix
from diffdrr.utils import get_focal_length
from diffdrr.utils import get_principal_point

for imod in range(0,1):
    drr = DRR(
    volume,  # The CT volume as a numpy array
    spacing,  # Voxel dimensions of the CT
    sdr=float(vsdr[imod]),  # Source-to-detector radius (half of the source-to-detector distance)
    height=256,  # Height of the DRR (if width is not seperately provided, the generated image is square)
    delx=2.0,  # Pixel spacing (in mm)
    ).to(device)

    thisIntrinsicMatrix=make_intrinsic_matrix(sdr=float(vsdr[imod]), delx=2.0, dely=2.0, height=256, width=256, x0=0.0, y0=0.0).to(device)
    ic(thisIntrinsicMatrix)

    thisFocalLength=get_focal_length (thisIntrinsicMatrix, delx=2.0, dely=2.0)
    ic(thisFocalLength)

    thisPrincipalPoint=get_principal_point (thisIntrinsicMatrix,  height=256, width=256, delx=2.0, dely=2.0)
    ic(thisPrincipalPoint)

  # Set the camera pose with rotations (yaw, pitch, roll) and translations (x, y, z)
    rotations = torch.tensor([[float(alpha[imod]), float(beta[imod]), torch.pi / 2]], device=device)
    translations = torch.tensor([[bx, by, bz]], device=device)
    img = drr(rotations, translations, parameterization="euler_angles", convention="ZYX")

    if(imod==0):
      Projection_mod0=torch.squeeze(img).cpu().numpy()
    if(imod==1):
      Projection_mod1=torch.squeeze(img).cpu().numpy()
    plot_drr(img)

    plt.title(f'Event{iEvent} plot_drr_mod0')
    plt.xlabel(f'Event{iEvent} TX={TX:5.1f} TY={TY:5.1f} TZ={TZ:5.1f}')
    plt.ylabel(f'Event{iEvent} Theta={ThetaDeg:.0f} Phi={PhiDeg:.0f}')
    plt.show()

From https://tutorial.pyvista.org/getting-started.html

In [ ]:
!apt-get install -qq xvfb libgl1-mesa-glx
!pip install pyvista -qq
!sudo apt install libgl1-mesa-glx xvfb

In [ ]:
import pyvista

pyvista.set_jupyter_backend('static')
pyvista.global_theme.notebook = True
pyvista.start_xvfb()

From https://vivekg.dev/DiffDRR/tutorials/visualization.html


In [ ]:
from diffdrr.visualization import drr_to_mesh, img_to_mesh

for imod in range(0,1):
  drr = DRR(
  volume,  # The CT volume as a numpy array
  spacing,  # Voxel dimensions of the CT
  sdr=float(vsdr[imod]),  # Source-to-detector radius (half of the source-to-detector distance)
  height=256,  # Height of the DRR (if width is not seperately provided, the generated image is square)
  delx=2.0,  # Pixel spacing (in mm)
  ).to(device)

  # Set the camera pose with rotations (yaw, pitch, roll) and translations (x, y, z)
  rotations = torch.tensor([[float(alpha[imod]), float(beta[imod]), torch.pi / 2]], device=device)
  ic(rotations)
  translations = torch.tensor([[bx, by, bz]], device=device)
  img = drr(rotations, translations, parameterization="euler_angles", convention="ZYX")

  # Make a mesh from the CT volume
  ct = drr_to_mesh(drr, "surface_nets", threshold=0.00001, verbose=False)
  #ct = drr_to_mesh(drr, "marching_cubes", threshold=0.0001, verbose=False)
  # Make the plot
  plotter = pyvista.Plotter()
  plotter.add_mesh(ct)
  # Make a mesh from the camera and detector plane
  pose = convert(rotations, translations, parameterization="euler_angles", convention="ZYX")
  camera, detector, texture, principal_ray = img_to_mesh(drr, pose)
  ic(principal_ray)

  # Make the plot
  plotter = pyvista.Plotter()
  plotter.add_mesh(ct)
  plotter.add_mesh(camera, show_edges=True, line_width=1.5)
  plotter.add_mesh(principal_ray, color="lime", line_width=3)
  plotter.add_mesh(detector, texture=texture)

  # Render the plot
  plotter.add_axes()
  plotter.add_bounding_box()
  plotter.show()
  #plotter.show(jupyter_backend="server")  # If running Jupyter remotely


Event0 mod11

In [ ]:
  for imod in range(5,6):

    drr = DRR(
    volume,  # The CT volume as a numpy array
    spacing,  # Voxel dimensions of the CT
    sdr=float(vsdr[imod]),  # Source-to-detector radius (half of the source-to-detector distance)
    height=256,  # Height of the DRR (if width is not seperately provided, the generated image is square)
    delx=2.0,  # Pixel spacing (in mm)
    ).to(device)

    # Set the camera pose with rotations (yaw, pitch, roll) and translations (x, y, z)
    rotations = torch.tensor([[float(alpha[imod]), float(beta[imod]), torch.pi / 2]], device=device)
    translations = torch.tensor([[bx, by, bz]], device=device)
    img = drr(rotations, translations, parameterization="euler_angles", convention="ZYX")

    if(imod==0):
      Projection_mod0=torch.squeeze(img).numpy()
    if(imod==1):
      Projection_mod1=torch.squeeze(img).numpy()
    plot_drr(img)

    plt.title(f'Event{iEvent} plot_drr_mod5')
    plt.xlabel(f'Event{iEvent} TX={TX:5.1f} TY={TY:5.1f} TZ={TZ:5.1f}')
    plt.ylabel(f'Event{iEvent} Theta={ThetaDeg:.0f} Phi={PhiDeg:.0f}')
    plt.show()

In [ ]:
for imod in range(5,6):
  drr = DRR(
  volume,  # The CT volume as a numpy array
  spacing,  # Voxel dimensions of the CT
  sdr=float(vsdr[imod]),  # Source-to-detector radius (half of the source-to-detector distance)
  height=256,  # Height of the DRR (if width is not seperately provided, the generated image is square)
  delx=2.0,  # Pixel spacing (in mm)
  ).to(device)

  # Set the camera pose with rotations (yaw, pitch, roll) and translations (x, y, z)
  rotations = torch.tensor([[float(alpha[imod]), float(beta[imod]), torch.pi / 2]], device=device)
  ic(rotations)
  translations = torch.tensor([[bx, by, bz]], device=device)
  img = drr(rotations, translations, parameterization="euler_angles", convention="ZYX")

  # Make a mesh from the CT volume
  ct = drr_to_mesh(drr, "surface_nets", threshold=0.00001, verbose=False)

  # Make the plot
  plotter = pyvista.Plotter()
  plotter.add_mesh(ct)
    # Make a mesh from the camera and detector plane
  pose = convert(rotations, translations, parameterization="euler_angles", convention="ZYX")
  camera, detector, texture, principal_ray = img_to_mesh(drr, pose)
  ic(principal_ray)

  # Make the plot
  plotter = pyvista.Plotter()
  plotter.add_mesh(ct)
  plotter.add_mesh(camera, show_edges=True, line_width=1.5)
  plotter.add_mesh(principal_ray, color="lime", line_width=3)
  plotter.add_mesh(detector, texture=texture)

  # Render the plot
  plotter.add_axes()
  plotter.add_bounding_box()
  plotter.show()